In [1]:
import pandas as pd
from cartoframes.data import enrichment
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.data.observatory import Catalog, Enrichment

## Search metadata

In [ ]:
catalog = Catalog()
countries = catalog.countries
countries

In [3]:
usa = countries.get('usa')
usa

<Country('usa')>

In [ ]:
datasets = usa.datasets
datasets

In [5]:
dataset = datasets.get('od_acs_d28e63ff')

In [ ]:
variables = dataset.variables
variables

In [7]:
variables_filtered = [variables.get('median_income_646ff1a1')]

In [8]:
variables_filtered

[<Variable('median_income_646ff1a1','Within a geographic area, the ...')>]

## Enrichment

In [9]:
credentials = Credentials.from_file()
set_default_credentials(credentials)


In [10]:
data = pd.read_csv('./data/points.csv')

In [11]:
data.head()

,NAME,BRANDNAME,PB_ID,TRADE_NAME,FRANCHISE_NAME,ISO3,AREANAME4,AREANAME3,AREANAME2,AREANAME1,...,GLOBAL_ULTIMATE_STREET_ADDRESS,GLOBAL_ULTIMATE_AREANAME3,GLOBAL_ULTIMATE_AREANAME1,GLOBAL_ULTIMATE_COUNTRY,GLOBAL_ULTIMATE_POSTCODE,FAMILY_MEMBERS,HIERARCHY_CODE,TICKER_SYMBOL,EXCHANGE_NAME,geom
0,HUSAIN'S FASHIONS,NaN,2011204802,NaN,NaN,USA,NaN,TUSCALOOSA,TUSCALOOSA,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-87.577749 33.202438)
1,GOODY'S,GOODY'S,2011227410,GOODY'S,NaN,USA,NaN,GENEVA,GENEVA,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-85.89445 31.047544)
2,CITI TRENDS,CITI TRENDS,2011234818,CITI TRENDS,NaN,USA,NaN,TALLADEGA,TALLADEGA,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-86.084328 33.43874)
3,DEPARTMENT STORE DIRECT,NaN,2011250243,NaN,NaN,USA,NaN,CULLMAN,CULLMAN,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-86.847353 34.185178)
4,SHELTONS CLOTHING,NaN,2011251945,NaN,NaN,USA,NaN,MOULTON,LAWRENCE,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-87.253304 34.464891)


In [12]:
# You may need access to Data Observatory for this
data_augmentated = Enrichment().enrich_points(data, variables_filtered, data_geom_column='geom')


                SELECT data_table.enrichment_id,
                    enrichment_table.median_income,
                    ST_Area(enrichment_geo_table.geom) AS demographics_acs_usa_blockgroupclipped_2015_5yrs_20062010_area
                FROM `carto-do-public-data.open_data.demographics_acs_usa_blockgroupclipped_2015_5yrs_20062010` enrichment_table
                    JOIN `carto-do-public-data.open_data.geography_usa_blockgroupclipped_2015` enrichment_geo_table
                        ON enrichment_table.geoid = enrichment_geo_table.geoid
                    JOIN `carto-do-customers.juanra_carto.temp_37a2be6995a74510972461a1a5defd37` data_table
                        ON ST_Within(data_table.geom, enrichment_geo_table.geom)
                ;
            


In [13]:
# Note the enrichment process also may include `area_XXX` columns from the original data geometries
data_augmentated.head()

,NAME,BRANDNAME,PB_ID,TRADE_NAME,FRANCHISE_NAME,ISO3,AREANAME4,AREANAME3,AREANAME2,AREANAME1,...,GLOBAL_ULTIMATE_AREANAME1,GLOBAL_ULTIMATE_COUNTRY,GLOBAL_ULTIMATE_POSTCODE,FAMILY_MEMBERS,HIERARCHY_CODE,TICKER_SYMBOL,EXCHANGE_NAME,geom,median_income,demographics_acs_usa_blockgroupclipped_2015_5yrs_20062010_area
0,HUSAIN'S FASHIONS,NaN,2011204802,NaN,NaN,USA,NaN,TUSCALOOSA,TUSCALOOSA,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.577749 33.202438),26000.0,5.100661e+14
1,HUSAIN'S FASHIONS,NaN,2011204802,NaN,NaN,USA,NaN,TUSCALOOSA,TUSCALOOSA,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.577749 33.202438),36638.0,5.100661e+14
2,HUSAIN'S FASHIONS,NaN,2011204802,NaN,NaN,USA,NaN,TUSCALOOSA,TUSCALOOSA,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.577749 33.202438),69341.0,5.100661e+14
3,HUSAIN'S FASHIONS,NaN,2011204802,NaN,NaN,USA,NaN,TUSCALOOSA,TUSCALOOSA,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.577749 33.202438),38750.0,5.100661e+14
4,HUSAIN'S FASHIONS,NaN,2011204802,NaN,NaN,USA,NaN,TUSCALOOSA,TUSCALOOSA,ALABAMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.577749 33.202438),56250.0,5.100661e+14
